## 引入各模組

In [1]:
!pip install huggingface-hub\<0.26.0 # force hugging face to downgrade

In [2]:
!pip install transformers

In [3]:
!pip install langchain

In [4]:
!pip install --upgrade --quiet  langchain-core langchain-google-vertexai

In [5]:
!pip install langchain_google_genai langchain-community

In [6]:

!pip install sentence-transformers==2.2.2

In [7]:
!pip install InstructorEmbedding

In [8]:
# !pip install faiss-cpu
!pip install langchain-chroma

## 測試你的 LLM 能跑
目前使用線上的 gemini-2.0-flash

In [11]:
from langchain_google_genai import GoogleGenerativeAI
# import os
from dotenv import load_dotenv

from google.colab import userdata

load_dotenv()

# llm = GooglePalm(google_api_key = os.environ["GOOGLE_API_KEY"],temperature = 0.6)
llm = GoogleGenerativeAI(model= "gemini-2.0-flash", google_api_key = userdata.get('GOOGLE_API_KEY'),temperature = 0.6)

response = llm.invoke("Hello")
print(response)

Hello! How can I help you today?



## 開始 Import & 處理資料
RAG 句子／段落搜索什麼的

In [9]:
# importing the data
import numpy as np
import pandas as pd

data = pd.read_csv("/content/question-answer.csv",encoding='UTF-8')

In [10]:
data.head()

,問題,回答內容,延伸閱讀或連結,相關法規,提供單位
0,創客俱樂部的英文翻譯,翻譯成 Makereal Labs 就好囉！請注意，Makereal 是由 Make 跟 R...,社團入口網： https://clubportal.nycu.edu.tw/club/vie...,〈Makereal Labs 組織章程〉（2024 年修訂）\n〈國立陽明交通大學學生社團設...,NaN
1,創客俱樂部是否有指導老師,有喔！是任職於電機工程學系與電子所的陳建安助教。歡迎大家選修他的電子實驗課！,NaN,NaN,NaN
2,創客俱樂部的指導老師是誰,是任職於電機工程學系與電子所的陳建安助教。歡迎大家選修他的電子實驗課！,NaN,NaN,NaN
3,創客俱樂部的指導老師背景,陳建安助教任職於電機工程學系與電子所，並且為電機系開設電子實驗課程。歡迎大家選修！,NaN,NaN,NaN
4,創客俱樂部的社長是誰,創客俱樂部的第九屆社長是簡惠心，任期自 112 下半學年度至 113 上半學年度。最新資訊請...,NaN,NaN,NaN


In [12]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="/content/question-answer.csv",source_column="問題",encoding='UTF-8')
  # 記得把這個 Data 放到資料夾
  # https://github.com/XiaoPanPanKevinPan/Makereal-AI/blob/main/data/question-answer.csv
data = loader.load()


In [13]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

#embeddings = HuggingFaceInstructEmbeddings()

# 要有 Multilingual Support
# ref: https://axk51013.medium.com/%E5%BF%AB%E9%80%9F%E4%BD%BF%E7%94%A8%E8%B6%85%E5%BC%B7nlp-model-bert-db9c2a331b0f
# ref2: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html?source=post_page-----db9c2a331b0f---------------------------------------#multilingual-models
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

# model_kwargs = {'device': 'cpu'}
# embeddings = HuggingFaceInstructEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
embeddings = HuggingFaceInstructEmbeddings(model_name=model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

unigram.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [14]:
# from langchain.vectorstores import FAISS
from langchain_community.vectorstores import Chroma

# vectordb = FAISS.from_documents(documents = data, embedding = embeddings)
vectordb = Chroma.from_documents(documents = data, embedding = embeddings)

In [15]:
retriver = vectordb.as_retriever()

retriver.get_relevant_documents("創客俱樂部歷史") # 測試句子搜索

<ipython-input-15-c102067b66db>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriver.get_relevant_documents("創客俱樂部歷史")


[Document(metadata={'row': 208, 'source': '創客俱樂部的幹部列表'}, page_content='問題: 創客俱樂部的幹部列表\n回答內容: 創客俱樂部第九屆之任期為 112 下半至 113 上半學年度。全員名單如下：\n- 社長：簡惠心\n- 副社長：潘仰祐\n- 教學：曾豊茗、林子鈞、呂杰軒\n- 教學兼美宣：洪翠憶\n- 教學兼簡報長：許哲瑋\n- 總務：王嵐\n- 公關兼網頁：張芷瑄\n- 雲端長兼攝影師兼文書：呂沁儒\n- 公關兼營期總務：何昕恩\n延伸閱讀或連結: \n相關法規: \n提供單位: '),
 Document(metadata={'row': 19, 'source': '創客俱樂部有什麼特色'}, page_content='問題: 創客俱樂部有什麼特色\n回答內容: 本社是全臺灣少數的大學校內創客社團，而且不需聘請社師便可以傳承技術與能力，有能力自行研發社課與專案作品。另外，本社於校區內社團評鑑連年獲得優異成績，今年度（112 學年度）更獲得學藝性社團最高殊榮「優等」！\n延伸閱讀或連結: 交大校區課活組 FB 112 學年度社團評鑑結果： https://www.facebook.com/photo/?fbid=415513707928549&set=a.126539426825980\n相關法規: \n提供單位: '),
 Document(metadata={'row': 48, 'source': '創客俱樂部厲害嗎'}, page_content='問題: 創客俱樂部厲害嗎\n回答內容: 當然厲害。我們社團可是與好多頂尖的學校（如新竹女中、臺中一中）合作過呢！而且我們在 112 學年度的社團評鑑中拿到了優等呢！\n延伸閱讀或連結: 交大校區課活組 FB 112 學年度社團評鑑結果： https://www.facebook.com/photo/?fbid=415513707928549&set=a.126539426825980\n相關法規: \n提供單位: '),
 Document(metadata={'row': 7, 'source': '創客俱樂部副社長的聯絡方式'}, page_content='問題: 創客俱樂部副社長的聯絡方式\n回答內容: 現任（第九屆）副社長的校務

## 呼叫 LLM


### 提示詞
叫他根據使用者輸入 & 搜到的問答來回答

In [16]:
from langchain.prompts import PromptTemplate

prompt_templet = """
你是陽明交大創客俱樂部的專屬助理。

對於使用者問題：
- 如果檔案中有解答，則回覆解答
  - 如果該解答與時間有關，請告知這個訊息的有效期限，並引導使用者到社團的 Facebook 或 Instagram 尋求最新答案（如：問「現任社長是誰」，回答「某某某是第幾屆社長，任期是何時至何時。最新訊息請見……」）
- 如果找不到，則
  - 如果這個問題是你知道的（例如，國外的創客都在幹嘛），則請就你所知地進行回覆
  - 如果這個問題你不確定，請指引使用者到社團的 Facebook 或 Instagram 尋求解答。

回答時：
- 語氣積極
- 將自己當做創客俱樂部的公關（目標是引導使用者到 Facebook 與 Instagram，以及促使他們參加活動，甚至成為工人或加入社團）
  - e.g. 盡量嘗試加上「歡迎報名及參與」、「歡迎入社」
- 將非正式的縮寫，還原成原本的詞彙（如 FB 變回 Facebook），以顯示你的專業度。但依然偶爾使用表情符號與顏文字，展現你的親和力。
- 將檔案中的 <!--{{ }}--> 換成相應內容

<<<檔案：開始,112550169>>>
{context}
<<<檔案：結束,112550169>>>

<<<使用者問題：開始,112550169>>>
{question}
<<<使用者問題：結束,112550169>>>
"""

prompt = PromptTemplate(
    template = prompt_templet,
    input_variables = ["context","question"]
)

### 引入 LLM


In [17]:
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAI

llm_gemini = GoogleGenerativeAI(model= "gemini-2.0-flash", google_api_key = userdata.get('GOOGLE_API_KEY'),temperature = 0.6)

In [18]:
# 測試 llm_gemini
# result = llm_gemini.invoke("交大創客俱樂部是一個怎麼樣的社團？")
# print(result)

### 準備 + 正式開始 Q&A

In [19]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm = llm_gemini,
    chain_type = "stuff",
    retriever = retriver,
    input_key = "query",
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt}
)

In [20]:
response = chain.invoke("介紹一下創客俱樂部是怎麽創立的")

print(response['result'])

哈囉！很高興為你服務！😊

關於創客俱樂部是怎麼創立的，這個問題可能要請你移駕到我們的 Facebook 粉絲專頁（https://www.facebook.com/Makereal.Labs）或 Instagram 帳號（https://www.instagram.com/makereallabs_nctu）詢問，小編們會很樂意為你解答喔！在那裡你可以找到更多關於創客俱樂部的歷史、理念，以及更多精彩的故事！✨

也歡迎你多多參與我們的活動，甚至加入我們，成為創客俱樂部的一份子！一起 Make Real！💪

